In [215]:
import pandas as pd
from scipy.stats import poisson
from sqlalchemy import and_

from models import Player, TradPlayerStats, AdvPlayerStats, Game
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()


In [238]:
def estimate_probability_poisson(data, stat, n):
    mean = data[stat].mean()
    probability = 1 - poisson.cdf(n, mean)
    return probability

def american_to_decimal(american_odds):
    """Convert American odds to decimal odds."""
    if american_odds > 0:
        return 1 + (american_odds / 100)
    else:
        return 1 + (100 / abs(american_odds))

def calculate_parlay_odds(american_odds_list):
    """Calculate the combined decimal odds for a parlay given a list of American odds."""
    decimal_odds = [american_to_decimal(odds) for odds in american_odds_list]
    
    # Calculate combined odds for the parlay
    combined_odds = 1
    for odds in decimal_odds:
        combined_odds *= odds
        
    return combined_odds

# Query to join four tables with correct join conditions and multiple filters
def get_and_save_player_data(player_id, player_name):
    data = session.query(
        Player,
        TradPlayerStats,
        AdvPlayerStats,
        Game
    ).join(Game, TradPlayerStats.game_id == Game.id)\
    .join(Player, TradPlayerStats.player_id == Player.id)\
    .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                                TradPlayerStats.player_id == AdvPlayerStats.player_id))\
    .filter(
        (TradPlayerStats.player_id == player_id)# &
        #  (Game.season_type == "Playoffs")
    ).all()

    # Convert the query result to a DataFrame
    data_list = []
    for player, trad_stats, adv_stats, game in data:
        row = {
            'player_name': player.name,
            'player_position': player.position,
            'minutes': trad_stats.minutes,
            'points': trad_stats.pts,
            'rebounds': trad_stats.reb,
            'assists': trad_stats.ast,
            'efg': adv_stats.efg_pct,
            'fg3a': trad_stats.fg3a,
            'fg3m': trad_stats.fg3m,
            'fg3_pct': trad_stats.fg3_pct,
            'fga': trad_stats.fga,
            'fgm': trad_stats.fgm,
            'fta': trad_stats.fta,
            'ft_pct': trad_stats.ft_pct, 
            'steals': trad_stats.stl,
            'blocks': trad_stats.blk,
            'date': game.date,

        }
        data_list.append(row)

    data_df = pd.DataFrame(data_list)

    data_df.to_csv(f"data_pile/{player_name}.csv")
    return data_df


def get_player_id(player_name):
    player = session.query(Player).filter(Player.name==player_name).all()[0]
    player_id = player.id
    return player_id


def calculate_ev(probability, decimal_odds, bet_amount):
    """
    Calculate the expected value (EV) of a bet.

    Args:
        probability (float): The probability of the event occurring (between 0 and 1).
        decimal_odds (float): The decimal odds for the bet.
        bet_amount (float): The amount wagered.

    Returns:
        float: The expected value (EV) of the bet.
    """
    payout = decimal_odds * bet_amount
    ev = (probability * payout) - bet_amount
    return ev


def implied_probability(decimal_odds):
    """Convert decimal odds to implied probability."""
    return 1 / decimal_odds


def estimate_probability_poisson_under(data, stat, n):
    mean = data[stat].mean()
    probability = poisson.cdf(n, mean)  # Calculate P(X <= n)
    return probability


def get_prop_probability(player_name, stat, prop_threshold, last_n_games=25, bet_type="over"):
    player_id = get_player_id(player_name)
    data = get_and_save_player_data(player_id, player_name).sort_values(by='date', ascending=False).head(last_n_games).copy()
    print(data.head())
    if bet_type == "over":
        return estimate_probability_poisson(data, stat, prop_threshold)
    elif bet_type == "under":
        return estimate_probability_poisson_under(data, stat, prop_threshold)
    else:
        raise ValueError("Invalid bet type. Use 'over' or 'under'.")
    

def analyze_bet(player_name, stat, threshold, odds, last_n_games, bet_type):
    probability = get_prop_probability(player_name, stat, threshold, last_n_games=last_n_games, bet_type=bet_type)
    odds = american_to_decimal(odds)
    house_probability = implied_probability(odds)
    ev = calculate_ev(probability, odds, 5)
    print(f"house_probability: {house_probability}")
    print(f"our probability: {probability}")
    print(f"odds: {odds}")
    print(f"ev: {ev}")
    return probability, odds


def calculate_combined_probability(probabilities):
    """Calculate the combined probability for a parlay."""
    combined_probability = 1
    for prob in probabilities:
        combined_probability *= prob
    return combined_probability


def calculate_combined_odds(decimal_odds_list):
    """Calculate the combined decimal odds for a parlay."""
    combined_odds = 1
    for odds in decimal_odds_list:
        combined_odds *= odds
    return combined_odds


def analyze_parlay(probabilities, odds_list, bet_amount):
    """
    Analyze a parlay bet and print the combined probability, combined odds, and expected value.

    Args:
        probabilities (list of float): The probabilities of individual bets.
        american_odds_list (list of int): The American odds for individual bets.
        bet_amount (float): The amount wagered.
    """
    combined_probability = calculate_combined_probability(probabilities)
    combined_odds = calculate_combined_odds(odds_list)
    ev = calculate_ev(combined_probability, combined_odds, bet_amount)
    
    print(f"Combined Probability: {combined_probability:.4f}")
    print(f"Combined Odds: {combined_odds:.2f}")
    print(f"Expected Value (EV): ${ev:.2f}")

    return ev

class Prop:
    def __init__(self, name, stat, threshold, odds, bet_type):
        self.name = name
        self.stat = stat
        self.n = threshold
        
        self.odds = odds
        self.bet_type = bet_type
        self.probability = self.get_prop_probability()
        self.ev, self.house_prob = self.get_ev_and_implied_prob()
        self.print_out = f"""
            PLAYER: {self.name}
              STAT: {self.stat}
            THRESH: {self.n}
              ODDS: {self.odds}
              TYPE: {self.bet_type}
              PROB: {self.probability}
                EV: {self.ev}
        HOUSE_PROB: {self.house_prob}
            """
        print(self.print_out)
        self.entry = {
            "PLAYER": self.name,
              "STAT": self.stat,
            "THRESH": self.n,
              "ODDS": self.odds,
              "TYPE": self.bet_type,
              "PROB": self.probability,
                "EV": self.ev,
        "HOUSE_PROB": {self.house_prob}
        }

    def get_prop_probability(self, last_n_games=25):
        player_id = get_player_id(self.name)
        data = get_and_save_player_data(player_id, self.name).sort_values(by='date', ascending=False).head(last_n_games).copy()
        # print(data.head())
        if self.bet_type == "over":
            return estimate_probability_poisson(data, self.stat, self.n)
        elif self.bet_type == "under":
            return estimate_probability_poisson_under(data, self.stat, self.n)
        else:
            raise ValueError("Invalid bet type. Use 'over' or 'under'.")
        
    def get_ev_and_implied_prob(self):
        odds = american_to_decimal(self.odds)
        house_probability = implied_probability(odds)
        ev = calculate_ev(self.probability, odds, 5)
        # print(f"house_probability: {house_probability}")
        # print(f"our probability: {self.probability}")
        # print(f"odds: {odds}")
        # print(f"ev: {ev}")
        return ev, house_probability
        
        
    
        

def analyze_parlay_list(parlay_list, bet_amount):
    probabilities = [prop.probability for prop in parlay_list]
    odds = [prop.odds for prop in parlay_list]
    return analyze_parlay(probabilities, odds, bet_amount)


def convert_series_to_dataframe(series, num_columns=5):
    # Initialize an empty list to store the rows
    rows = []

    # Process the series in chunks of 5
    for i in range(0, len(series), num_columns):
        row = series.iloc[i:i+num_columns].tolist()
        if len(row) == num_columns:
            rows.append(row)
    
    # Create a DataFrame from the rows
    df = pd.DataFrame(rows, columns=['Player', 'O_Line', 'O_Odds', 'U_Line', 'U_Odds'])
    return df


def convert_to_dataframe(data):
    # Split the input data by new lines
    lines = data.split('\n')
    
    # Initialize an empty list to store the rows
    rows = []

    # Process the lines in chunks of 5
    for i in range(0, len(lines), 5):
        row = lines[i:i+5]
        if len(row) == 5:
            rows.append(row)
    
    # Create a DataFrame from the rows
    df = pd.DataFrame(rows, columns=['Player', 'O_Line', 'O_Odds', 'U_Line', 'U_Odds'])
    return df
    

In [217]:
props = []

In [233]:
props_series = pd.read_csv("prop_lines/prop_lines.csv")
stat = props_series.columns[0]
props_df = convert_series_to_dataframe(props_series[stat])
for _, row in props_df.iterrows():
    prop_over = Prop(row['Player'], stat, float(row['O_Line'].split(" ")[1]), int(row["O_Odds"]), "over")
    props.append(prop_over)
    prop_under = Prop(row['Player'], stat, float(row['U_Line'].split(" ")[1]), int(row["U_Odds"]), "under")
    props.append(prop_under)





            PLAYER: Anthony Edwards
              STAT: blocks
            THRESH: 0.5
              ODDS: 105
              TYPE: over
              PROB: 0.428790936151185
                EV: -0.6048929044503533
        HOUSE_PROB: 0.48780487804878053
            

            PLAYER: Anthony Edwards
              STAT: blocks
            THRESH: 0.5
              ODDS: -140
              TYPE: under
              PROB: 0.571209063848815
                EV: -0.10392230986729967
        HOUSE_PROB: 0.5833333333333333
            

            PLAYER: Jaden McDaniels
              STAT: blocks
            THRESH: 0.5
              ODDS: -140
              TYPE: over
              PROB: 0.5852170883184187
                EV: 0.016146471300732657
        HOUSE_PROB: 0.5833333333333333
            

            PLAYER: Jaden McDaniels
              STAT: blocks
            THRESH: 0.5
              ODDS: 110
              TYPE: under
              PROB: 0.4147829116815813
               

In [275]:
good_props = [prop for prop in props if prop.ev > 0]
good_props_rows = pd.DataFrame.from_dict([prop.entry for prop in good_props])

In [277]:
good_props_rows = good_props_rows[good_props_rows['PLAYER'] != "Naz Reid"]
good_props_rows = good_props_rows[good_props_rows['PLAYER'] != "Mike Conley"]
good_props_rows = good_props_rows[good_props_rows['PLAYER'] != "Karl-Anthony Towns"]
good_props_rows = good_props_rows.sort_values('EV', ascending=False).head(32)
print(len(good_props_rows))
print(len(props))

19
168


In [278]:
teams = dm.query_teams()
players = dm.query_players()
player_to_team_map = {}
for player in players:
    team_id = player.team_id
    team_name = [team.full_name for team in teams if team.id == team_id][0]
    player_to_team_map[player.name] = team_name
teams = []
for _, row in good_props_rows.iterrows():
    teams.append(player_to_team_map[row['PLAYER']])
good_props_rows["TEAMS"] = pd.Series(teams)
good_props_rows.to_csv("good_props.csv")


In [279]:
df = pd.read_csv("good_props.csv", index_col = 0)
# df[df['PLAYER'] == "Naz Reid"] = np.nan
# df = df.dropna()

In [280]:
print(df)
print(len(df))

                      PLAYER      STAT  THRESH  ODDS   TYPE      PROB  \
3         Michael Porter Jr.    points    12.5  -130   over  0.874754   
5                Rudy Gobert    points    11.5  -120   over  0.769153   
61              Nikola Jokic    blocks     0.5   165  under  0.527292   
42        Michael Porter Jr.       fgm     4.5  -130   over  0.760249   
2   Kentavious Caldwell-Pope    points     7.5  -125   over  0.741572   
8            Anthony Edwards    points    29.5  -125  under  0.738896   
1               Jamal Murray    points    19.5  -110   over  0.691024   
22  Kentavious Caldwell-Pope      fg3m     1.5   120   over  0.571908   
53              Aaron Gordon    steals     1.5  -225  under  0.864760   
15           Anthony Edwards   assists     6.5  -115  under  0.657552   
11              Jamal Murray   assists     5.5  -115   over  0.645327   
45               Rudy Gobert       fgm     4.5  -110   over  0.626689   
34           Christian Braun  rebounds     3.5   13

In [281]:
import itertools


n = 7

# Generate all combinations of n rows
combinations = list(itertools.combinations(df.index, n))

# Function to evaluate heterogeneity of a combination
def evaluate_heterogeneity(comb, df):
    comb_list = list(comb)
    players = df.loc[comb_list, 'PLAYER']
    stats = df.loc[comb_list, 'STAT']
    teams = df.loc[comb_list, 'TEAMS']
    # Calculate a simple heterogeneity score (you can define your own logic)
    player_score = len(set(players))
    stat_score = len(set(stats))
    team_score = len(set(teams))
    return player_score + stat_score + team_score

# Evaluate all combinations and sort them by heterogeneity score
comb_scores = [(comb, evaluate_heterogeneity(comb, df)) for comb in combinations]
comb_scores_sorted = sorted(comb_scores, key=lambda x: x[1], reverse=True)

# Select the most heterogeneous combinations (you can define how many you want)
top_combinations = comb_scores_sorted[:5]  # Top 5 combinations for example

# Display the most heterogeneous combinations
for comb, score in top_combinations:
    print(f"Combination: {comb}, Score: {score}")
    print(df.loc[list(comb)])
    print()

# Optional: Convert combinations to DataFrame
top_comb_dfs = [df.loc[list(comb)] for comb, score in top_combinations]

Combination: (3, 61, 22, 53, 15, 45, 34), Score: 17
                      PLAYER      STAT  THRESH  ODDS   TYPE      PROB  \
3         Michael Porter Jr.    points    12.5  -130   over  0.874754   
61              Nikola Jokic    blocks     0.5   165  under  0.527292   
22  Kentavious Caldwell-Pope      fg3m     1.5   120   over  0.571908   
53              Aaron Gordon    steals     1.5  -225  under  0.864760   
15           Anthony Edwards   assists     6.5  -115  under  0.657552   
45               Rudy Gobert       fgm     4.5  -110   over  0.626689   
34           Christian Braun  rebounds     3.5   130   over  0.518298   

          EV             HOUSE_PROB                   TEAMS  
3   2.738205   {0.5652173913043479}          Denver Nuggets  
61  1.986625  {0.37735849056603776}                     NaN  
22  1.290984  {0.45454545454545453}                     NaN  
53  1.245486   {0.6923076923076923}                     NaN  
15  1.146685   {0.5348837209302326}  Minnesota Timber

In [284]:
from datetime import date
import random

# Generate a random UUID (Universally Unique Identifier)


for i, df in enumerate(top_comb_dfs):
    random_id = random.randint(100000, 999999)
    probs = list(df['PROB'])
    odds = [american_to_decimal(odds) for odds in list(df['ODDS'])]
    df['COMBINED EV'] = analyze_parlay(probs, odds, 5)
    df['COMBINED PROB'] = calculate_combined_probability(probs)
    df.to_csv(f"parlays/{date.today()}_{random_id}_{i}.csv")

Combined Probability: 0.0487
Combined Odds: 122.31
Expected Value (EV): $24.79
Combined Probability: 0.0699
Combined Odds: 84.46
Expected Value (EV): $24.53
Combined Probability: 0.0478
Combined Odds: 122.31
Expected Value (EV): $24.24
Combined Probability: 0.0686
Combined Odds: 84.46
Expected Value (EV): $23.98
Combined Probability: 0.0490
Combined Odds: 115.32
Expected Value (EV): $23.25


In [204]:
print(good_props_rows.iloc[0:1])

         PLAYER    STAT  THRESH  ODDS   TYPE      PROB        EV  \
0  Nikola Jokic  points    29.5  -120  under  0.587523  0.385625   

             HOUSE_PROB           TEAMS  
0  {0.5454545454545454}  Denver Nuggets  


In [ ]:
name, prop = "Kentavious Caldwell-Pope,110 over 2.5 assists".split(",")
prop = prop.split(" ")
prop = Prop(name, prop[3].strip(),  float(prop[2]), int(prop[0]), prop[1].strip())
if prop.ev > 0:
    print(prop.ev)
    props.append(prop)
    print("PROP ADDED")
print([prop.name for prop in props])



In [ ]:
props.pop()
print([prop.name for prop in props])

In [ ]:
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]

In [ ]:
parlay.pop()
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]

In [ ]:
prop_to_add = Prop(player_name, stat, threshold, probability, odds, bet_type)
parlay.append(prop_to_add)
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]



In [ ]:
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]
print(analyze_parlay_list(parlay, 5))